In [3]:
!pip install --upgrade pip setuptools


  Using cached setuptools-75.1.0-py3-none-any.whl.metadata (6.9 kB)
Using cached setuptools-75.1.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 56.0.0
    Uninstalling setuptools-56.0.0:
      Successfully uninstalled setuptools-56.0.0


In [14]:
!pip install groq

  Using cached groq-0.11.0-py3-none-any.whl.metadata (13 kB)
Using cached groq-0.11.0-py3-none-any.whl (106 kB)


In [4]:
!pip install python-dotenv


In [17]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 3.3 MB/s eta 0:00:00a 0:00:01


In [1]:
# Install all needed packages
%pip install numpy openai weave asyncio


  Using cached asyncio-3.4.3-py3-none-any.whl.metadata (1.7 kB)
  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [29 lines of output]
        Using cached distribute-0.7.3.zip (145 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        × Preparing metadata (pyproject.toml) did not run successfully.
        │ exit code: 1
        ╰─> [6 lines of output]
  

In [18]:
#Load API from .env file for open AI and also Weights & Biases
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
wandb_api_key = os.getenv("WANDB_API_KEY")

if not openai_api_key or not wandb_api_key:
    raise ValueError("API keys not found in .env file")
        

## 1. Build a knowledge base

In [19]:
from openai import OpenAI
import weave
from weave import Model
import numpy as np
import json
import asyncio 
import faiss
import os
from dotenv import load_dotenv

load_dotenv()

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

articles = [
    "Novo Nordisk and Eli Lilly rival soars 32 percent after promising weight loss drug results Shares of Denmarks Zealand Pharma shot 32 percent higher in morning trade, after results showed success in its liver disease treatment survodutide, which is also on trial as a drug to treat obesity. The trial \"tells us that the 6mg dose is safe, which is the top dose used in the ongoing [Phase 3] obesity trial too,\" one analyst said in a note. The results come amid feverish investor interest in drugs that can be used for weight loss.",
    "Berkshire shares jump after big profit gain as Buffetts conglomerate nears $1 trillion valuation Berkshire Hathaway shares rose on Monday after Warren Buffetts conglomerate posted strong earnings for the fourth quarter over the weekend. Berkshires Class A and B shares jumped more than 1.5%, each. Class A shares are higher by more than 17% this year, while Class B have gained more than 18%. Berkshire was last valued at $930.1 billion, up from $905.5 billion where it closed on Friday, according to FactSet. Berkshire on Saturday posted fourth-quarter operating earnings of $8.481 billion, about 28 percent higher than the $6.625 billion from the year-ago period, driven by big gains in its insurance business. Operating earnings refers to profits from businesses across insurance, railroads and utilities. Meanwhile, Berkshires cash levels also swelled to record levels. The conglomerate held $167.6 billion in cash in the fourth quarter, surpassing the $157.2 billion record the conglomerate held in the prior quarter.",
    "Highmark Health says its combining tech from Google and Epic to give doctors easier access to information Highmark Health announced it is integrating technology from Google Cloud and the health-care software company Epic Systems. The integration aims to make it easier for both payers and providers to access key information they need, even if its stored across multiple points and formats, the company said. Highmark is the parent company of a health plan with 7 million members, a provider network of 14 hospitals and other entities",
    "Rivian and Lucid shares plunge after weak EV earnings reports Shares of electric vehicle makers Rivian and Lucid fell Thursday after the companies reported stagnant production in their fourth-quarter earnings after the bell Wednesday. Rivian shares sank about 25 percent, and Lucids stock dropped around 17 percent. Rivian forecast it will make 57,000 vehicles in 2024, slightly less than the 57,232 vehicles it produced in 2023. Lucid said it expects to make 9,000 vehicles in 2024, more than the 8,428 vehicles it made in 2023.",
    "Mauritius blocks Norwegian cruise ship over fears of a potential cholera outbreak Local authorities on Sunday denied permission for the Norwegian Dawn ship, which has 2,184 passengers and 1,026 crew on board, to access the Mauritius capital of Port Louis, citing \"potential health risks.\" The Mauritius Ports Authority said Sunday that samples were taken from at least 15 passengers on board the cruise ship. A spokesperson for the U.S.-headquartered Norwegian Cruise Line Holdings said Sunday that 'a small number of guests experienced mild symptoms of a stomach-related illness' during Norwegian Dawns South Africa voyage.",
    "Intuitive Machines lands on the moon in historic first for a U.S. company Intuitive Machines Nova-C cargo lander, named Odysseus after the mythological Greek hero, is the first U.S. spacecraft to soft land on the lunar surface since 1972. Intuitive Machines is the first company to pull off a moon landing — government agencies have carried out all previously successful missions. The companys stock surged in extended trading Thursday, after falling 11 percent in regular trading.",
    "Lunar landing photos: Intuitive Machines Odysseus sends back first images from the moon Intuitive Machines cargo moon lander Odysseus returned its first images from the surface. Company executives believe the lander caught its landing gear sideways in the moons surface while touching down and tipped over. Despite resting on its side, the companys historic IM-1 mission is still operating on the moon.",
]

def create_embeddings(texts):
    embeddings = []
    for text in texts:
        response = openai_client.embeddings.create(
            input=text,
            model="text-embedding-3-small"
        )
        embeddings.append(response.data[0].embedding)
    return np.array(embeddings, dtype=np.float32)

def create_rag_database(articles):
    embeddings = create_embeddings(articles)
    
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    
    faiss.write_index(index, "article_index.faiss")
    
    with open("articles.json", "w") as f:
        json.dump(articles, f)

    print("RAG database created successfully.")

create_rag_database(articles)

def query_rag_database(query):
    index = faiss.read_index("article_index.faiss")
    
    with open("articles.json", "r") as f:
        stored_articles = json.load(f)
    
    query_embedding = create_embeddings([query])[0]
    
    k = 1
    distances, indices = index.search(np.array([query_embedding]), k)
    
    most_relevant_doc = stored_articles[indices[0][0]]
    
    return most_relevant_doc

class RAGModel(Model):
    system_message: str
    model_name: str = "gpt-4o"

    @weave.op()
    def predict(self, question: str) -> dict:
        context = query_rag_database(question)
        client = OpenAI()
        query = f"""Use the following information to answer the subsequent question. If the answer cannot be found, write "I don't know."
        Context:
        \"\"\"
        {context}
        \"\"\"
        Question: {question}"""
        response = client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": self.system_message},
                {"role": "user", "content": query},
            ],
            temperature=0.0,
            response_format={"type": "text"},
        )
        answer = response.choices[0].message.content
        return {'answer': answer, 'context': context}

RAG database created successfully.


## Create a RAG App

In [20]:
import faiss
import numpy as np
import json
from openai import OpenAI
import weave
from weave import Model
import os
from dotenv import load_dotenv

load_dotenv()

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def create_embeddings(texts):
    embeddings = []
    for text in texts:
        response = openai_client.embeddings.create(
            input=text,
            model="text-embedding-3-small"
        )
        embeddings.append(response.data[0].embedding)
    return np.array(embeddings, dtype=np.float32)

def query_rag_database(query):
    index = faiss.read_index("article_index.faiss")
    
    with open("articles.json", "r") as f:
        stored_articles = json.load(f)
    
    query_embedding = create_embeddings([query])[0]
    
    k = 1  # Number of similar documents to retrieve
    distances, indices = index.search(np.array([query_embedding]), k)
    
    most_relevant_doc = stored_articles[indices[0][0]]
    
    return most_relevant_doc

class RAGModel(Model):
    system_message: str
    model_name: str = "gpt-4-turbo-preview"  # Updated to a more recent model

    @weave.op()
    def predict(self, question: str) -> dict:
        context = query_rag_database(question)
        client = OpenAI()
        query = f"""Use the following information to answer the subsequent question. If the answer cannot be found, write "I don't know."
        Context:
        \"\"\"
        {context}
        \"\"\"
        Question: {question}"""
        response = client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": self.system_message},
                {"role": "user", "content": query},
            ],
            temperature=0.0,
            response_format={"type": "text"},
        )
        answer = response.choices[0].message.content
        return {'answer': answer, 'context': context}

weave.init('rag-qa')
model = RAGModel(
    system_message="You are an expert in finance and answer questions related to finance, financial services, and financial markets. When responding based on provided information, be sure to cite the source."
)

# Test the model
result = model.predict("What significant result was reported about Zealand Pharma's obesity trial?")
print(result)

🍩 https://wandb.ai/tuminha/rag-qa/r/call/01925065-a9d9-7773-8249-4c385c039b32
{'answer': "The significant result reported about Zealand Pharma's obesity trial was the success in its liver disease treatment survodutide, which is also on trial as a drug to treat obesity. The trial indicated that the 6mg dose is safe, which is the top dose used in the ongoing Phase 3 obesity trial.", 'context': 'Novo Nordisk and Eli Lilly rival soars 32 percent after promising weight loss drug results Shares of Denmarks Zealand Pharma shot 32 percent higher in morning trade, after results showed success in its liver disease treatment survodutide, which is also on trial as a drug to treat obesity. The trial "tells us that the 6mg dose is safe, which is the top dose used in the ongoing [Phase 3] obesity trial too," one analyst said in a note. The results come amid feverish investor interest in drugs that can be used for weight loss.'}


## 3. Evaluating with an LLM Judge

When there aren't simple ways to evaluate your application, one approach is to use an LLM to evaluate aspects of it. Here is an example of using an LLM judge to try to measure the context precision by prompting it to verify if the context was useful in arriving at the given answer. This prompt was augmented from the popular RAGAS framework.

### Defining a scoring function

As we did in the Build an Evaluation pipeline tutorial, we'll define a set of example rows to test our app against and a scoring function. Our scoring function will take one row and evaluate it. The input arguments should match with the corresponding keys in our row, so question here will be taken from the row dictionary. model_output is the output of the model. The input to the model will be taken from the example based on its input argument, so question here too. We're using async functions so they run fast in parallel. If you need a quick introduction to async, you can find one here [https://realpython.com/async-io-python/](https://realpython.com/async-io-python/).

In [21]:
import faiss
import numpy as np
import json
from openai import OpenAI
import weave
import asyncio
from groq import Groq
import os
import nest_asyncio

# This allows asyncio.run() to work in Jupyter notebooks
nest_asyncio.apply()

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def create_embeddings(texts):
    embeddings = []
    for text in texts:
        response = openai_client.embeddings.create(
            input=text,
            model="text-embedding-3-small"
        )
        embeddings.append(response.data[0].embedding)
    return np.array(embeddings, dtype=np.float32)

def query_rag_database(query):
    index = faiss.read_index("article_index.faiss")
    
    with open("articles.json", "r") as f:
        stored_articles = json.load(f)
    
    query_embedding = create_embeddings([query])[0]
    
    k = 1  # Number of similar documents to retrieve
    distances, indices = index.search(np.array([query_embedding]), k)
    
    most_relevant_doc = stored_articles[indices[0][0]]
    
    return most_relevant_doc

@weave.op()
async def context_precision_score(question, model_output):
    context_precision_prompt = """Given question, answer and context verify if the context was useful in arriving at the given answer. Give verdict as "1" if useful and "0" if not with json output. 
    Output in only valid JSON format.

    question: {question}
    context: {context}
    answer: {answer}
    verdict: """
    
    prompt = context_precision_prompt.format(
        question=question,
        context=model_output['context'],
        answer=model_output['answer'],
    )

    response = openai_client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[{"role": "user", "content": prompt}],
        response_format={ "type": "json_object" }
    )
    response_message = response.choices[0].message
    response = json.loads(response_message.content)
    return {
        "verdict": int(response["verdict"]) == 1,
    }

questions = [
    {"question": "What significant result was reported about Zealand Pharma's obesity trial?"},
    {"question": "How much did Berkshire Hathaway's cash levels increase in the fourth quarter?"},
    {"question": "What is the goal of Highmark Health's integration of Google Cloud and Epic Systems technology?"},
    {"question": "What were Rivian and Lucid's vehicle production forecasts for 2024?"},
    {"question": "Why was the Norwegian Dawn cruise ship denied access to Mauritius?"},
    {"question": "Which company achieved the first U.S. moon landing since 1972?"},
    {"question": "What issue did Intuitive Machines' lunar lander encounter upon landing on the moon?"}
]

@weave.op()
async def rag_groq_model(question):
    context = query_rag_database(question)
    
    prompt = f"""Use the following information to answer the subsequent question. If the answer cannot be found, write "I don't know."
    Context:
    \"\"\"
    {context}
    \"\"\"
    Question: {question}"""
    
    response = groq_client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        model="mixtral-8x7b-32768",
        temperature=0.5,
        max_tokens=150
    )
    return {
        "answer": response.choices[0].message.content,
        "context": context
    }

# Create the evaluation
evaluation = weave.Evaluation(dataset=questions, scorers=[context_precision_score])

# Run the evaluation
async def run_evaluation():
    results = await evaluation.evaluate(rag_groq_model)
    print(results)

# Run the evaluation
asyncio.run(run_evaluation())

Evaluated 1 of 7 examples

Evaluated 2 of 7 examples

Evaluated 3 of 7 examples

Evaluated 4 of 7 examples

Evaluated 5 of 7 examples

Evaluated 6 of 7 examples

Evaluated 7 of 7 examples

Evaluation summary
{
    'context_precision_score': {'verdict': {'true_count': 7, 'true_fraction': 1.0}},
    'model_latency': {'mean': 0.6660459041595459}
}

🍩 https://wandb.ai/tuminha/rag-qa/r/call/01925067-22a0-7011-9d48-db03ef7c717e
{'context_precision_score': {'verdict': {'true_count': 7, 'true_fraction': 1.0}}, 'model_latency': {'mean': 0.6660459041595459}}


## Defining a Scorer class

In some applications we want to create custom evaluation classes - where for example a standardized LLMJudge class should be created with specific parameters (e.g. chat model, prompt), specific scoring of each row, and specific calculation of an aggregate score. In order to do that Weave defines a list of ready-to-use Scorer classes and also makes it easy to create a custom Scorer - in the following we'll see how to create a custom class CorrectnessLLMJudge(Scorer).

On a high-level the steps to create custom Scorer are quite simple:

Define a custom class that inherits from weave.flow.scorer.Scorer
Overwrite the score function and add a @weave.op()if you want to track each call of the function
this function has to define a model_output argument where the prediction of the model will be passed to. We define it as type Optional[dict] in case the mode might return "None".
the rest of the arguments can either be a general Any or dict or can select specific columns from the dataset that is used to evaluate the model using the weave.Evaluate class - they have to have the exact same names as the column names or keys of a single row after being passed to preprocess_model_input if that is used.
Optional: Overwrite the summarize function to customize the calculation of the aggregate score. By default Weave uses the weave.flow.scorer.auto_summarizefunction if you don't define a custom function.
this function has to have a @weave.op() decorator.

In [23]:
import weave
from weave.flow.scorer import Scorer
from typing import Optional, Any, List
import numpy as np
from openai import OpenAI
import os
import json
import asyncio
from dotenv import load_dotenv

load_dotenv()

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

class RAGCorrectnessLLMJudge(Scorer):
    prompt: str = """Given the question, context, and answer, evaluate if the answer is correct and relevant based on the provided context. 
    Respond with a JSON object containing two keys:
    1. "correct": true if the answer is factually correct and relevant to the question, false otherwise.
    2. "explanation": a brief explanation of your evaluation.

    Question: {question}
    Context: {context}
    Answer: {answer}

    JSON response:"""
    model_name: str = "gpt-4-turbo-preview"

    @weave.op()
    async def score(self, model_output: Optional[dict], question: str) -> Any:
        """Score the correctness of the predictions by comparing the question, context, and answer."""
        if model_output is None or 'answer' not in model_output or 'context' not in model_output:
            return {"correct": False, "explanation": "Invalid model output"}

        eval_prompt = self.prompt.format(
            question=question,
            context=model_output['context'],
            answer=model_output['answer']
        )

        response = openai_client.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": eval_prompt}],
            response_format={"type": "json_object"}
        )

        evaluation = response.choices[0].message.content
        evaluation_dict = json.loads(evaluation)

        return {
            "correct": evaluation_dict["correct"],
            "explanation": evaluation_dict["explanation"]
        }

    @weave.op()
    def summarize(self, score_rows: List[dict]) -> Optional[dict]:
        """Aggregate all the scores calculated for each row by the scoring function."""
        valid_data = [x.get("correct") for x in score_rows if x.get("correct") is not None]
        count_true = list(valid_data).count(True)
        int_data = [int(x) for x in valid_data]

        sample_mean = np.mean(int_data) if int_data else 0
        sample_variance = np.var(int_data) if int_data else 0
        sample_error = np.sqrt(sample_variance / len(int_data)) if int_data else 0

        return {
            "correct": {
                "true_count": count_true,
                "true_fraction": sample_mean,
                "stderr": sample_error,
            }
        }

# Create an instance of our custom scorer
rag_correctness_judge = RAGCorrectnessLLMJudge()

# Create the evaluation with only our custom scorer
evaluation = weave.Evaluation(dataset=questions, scorers=[rag_correctness_judge])

# Run the evaluation
async def run_evaluation():
    results = await evaluation.evaluate(rag_groq_model)
    print(results)

# Run the evaluation
asyncio.run(run_evaluation())

Evaluated 1 of 7 examples

Evaluated 2 of 7 examples

Evaluated 3 of 7 examples

Evaluated 4 of 7 examples

Evaluated 5 of 7 examples

Evaluated 6 of 7 examples

Evaluated 7 of 7 examples

Evaluation summary
{
    'RAGCorrectnessLLMJudge': {'correct': {'true_count': 7, 'true_fraction': 1.0, 'stderr': 0.0}},
    'model_latency': {'mean': 0.6492925371442523}
}

🍩 https://wandb.ai/tuminha/rag-qa/r/call/01925074-2551-7d53-8f0d-1bd569e44f26
{'RAGCorrectnessLLMJudge': {'correct': {'true_count': 7, 'true_fraction': 1.0, 'stderr': 0.0}}, 'model_latency': {'mean': 0.6492925371442523}}


## 4. Pulling it all together

### To get the same result for your RAG apps:

Wrap LLM calls & retrieval step functions with weave.op()
(optional) Create a Model subclass with predict function and app details
Collect examples to evaluate
Create scoring functions that score one example
Use Evaluation class to run evaluations on your examples
NOTE: Sometimes the async execution of Evaluations will trigger a rate limit on the models of OpenAI, Anthropic, etc. To prevent that you can set an environment variable to limit the amount of parallel workers e.g. WEAVE_PARALLELISM=3.

Here, we show the code in it's entirety.

In [25]:
from groq import Groq

class RAGModel(Model):
    system_message: str
    model_name: str = "mixtral-8x7b-32768"  # Groq model

    @weave.op()
    def predict(self, question: str) -> dict:
        context = query_rag_database(question)
        groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))
        query = f"""Use the following information to answer the subsequent question. If the answer cannot be found, write "I don't know."
        Context:
        \"\"\"
        {context}
        \"\"\"
        Question: {question}"""
        response = groq_client.chat.completions.create(
            messages=[
                {"role": "system", "content": self.system_message},
                {"role": "user", "content": query},
            ],
            model=self.model_name,
            temperature=0.0,
            max_tokens=150
        )
        answer = response.choices[0].message.content
        return {'answer': answer, 'context': context}

weave.init('rag-qa')
model = RAGModel(
    system_message="You are an expert in finance and answer questions related to finance, financial services, and financial markets. When responding based on provided information, be sure to cite the source."
)

@weave.op()
async def context_precision_score(question, model_output):
    context_precision_prompt = """Given question, answer and context verify if the context was useful in arriving at the given answer. Give verdict as "1" if useful and "0" if not with json output. 
    Output in only valid JSON format.

    question: {question}
    context: {context}
    answer: {answer}
    verdict: """
    client = OpenAI()

    prompt = context_precision_prompt.format(
        question=question,
        context=model_output['context'],
        answer=model_output['answer'],
    )

    response = client.chat.completions.create(
        model="gpt-4o",  # Changed to gpt-4o
        messages=[{"role": "user", "content": prompt}],
        response_format={ "type": "json_object" }
    )
    response_message = response.choices[0].message
    response = json.loads(response_message.content)
    return {
        "verdict": int(response["verdict"]) == 1,
    }

evaluation = weave.Evaluation(dataset=questions, scorers=[context_precision_score])

async def run_evaluation():
    results = await evaluation.evaluate(model)
    print(results)

asyncio.run(run_evaluation())

Evaluated 1 of 7 examples

Evaluated 2 of 7 examples

Evaluated 3 of 7 examples

Evaluated 4 of 7 examples

Evaluated 5 of 7 examples

Evaluated 6 of 7 examples

Evaluated 7 of 7 examples

Evaluation summary
{
    'context_precision_score': {'verdict': {'true_count': 7, 'true_fraction': 1.0}},
    'model_latency': {'mean': 3.173034497669765}
}

🍩 https://wandb.ai/tuminha/rag-qa/r/call/0192507d-8ab3-78f0-ba11-d1ed54f28be5
{'context_precision_score': {'verdict': {'true_count': 7, 'true_fraction': 1.0}}, 'model_latency': {'mean': 3.173034497669765}}
